# Chess Engine with PyTorch

## Imports

In [12]:
import os
import numpy as np 
import time
import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader     
from chess import pgn 
from tqdm import tqdm 
import chess
from auxiliary_func import create_input_for_nn, encode_moves
from dataset import ChessDataset
from model import ChessModel
from multiprocessing import Pool, cpu_count
import matplotlib.pyplot as plt

# Data preprocessing

## Load data

In [2]:
def load_pgn(file_path, limit_per_file=None):
    games = []
    with open(file_path, 'r') as pgn_file:
        count = 0
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
            count += 1
            if limit_per_file is not None and count >= limit_per_file:
                break
    return games

# Lister tous les fichiers PGN
files = [file for file in os.listdir("../data/pgn") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 28)
LIMIT_PER_FILE = 500  # Limite de parties par fichier

# Charger les fichiers avec une barre de progression
games = []
for i, file in enumerate(tqdm(files[:LIMIT_OF_FILES], desc="Loading PGN files")):
    games.extend(load_pgn(f"../data/pgn/{file}", limit_per_file=LIMIT_PER_FILE))

print(f"GAMES PARSED: {len(games)}")

Loading PGN files: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

GAMES PARSED: 500


In [3]:
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 500


## Convert data into tensors

In [4]:
X, y = create_input_for_nn(games)

print(f"NUMBER OF SAMPLES: {len(y)}")

X = X[0:2500000]
y = y[0:2500000]

y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

NUMBER OF SAMPLES: 43532


# Preliminary actions

In [ ]:
# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=cpu_count())

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cuda


# Training

In [ ]:
def train(model, dataloader, optimizer, criterion, device, num_epochs=50):
    model.to(device)
    epoch_losses = []

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0

        # tqdm avec affichage de l'epoch et progression fine
        pbar = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)
        for inputs, labels in pbar:
            # transfert non‐bloquant si possible
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (pbar.n + 1), refresh=False)

        avg_loss = running_loss / len(dataloader)
        epoch_losses.append(avg_loss)
        print(f"Epoch {epoch}/{num_epochs} — avg loss: {avg_loss:.4f}")

    # Tracé de la courbe de perte
    plt.figure(figsize=(6,4))
    plt.plot(range(1, num_epochs+1), epoch_losses, marker='o')
    plt.title("Training Loss over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return epoch_losses


100%|██████████| 681/681 [00:02<00:00, 279.58it/s]


Epoch 1/50, Loss: 6.5023, Time: 0m2s


100%|██████████| 681/681 [00:02<00:00, 312.99it/s]


Epoch 2/50, Loss: 6.0527, Time: 0m2s


100%|██████████| 681/681 [00:02<00:00, 313.88it/s]


...

Epoch 48/50, Loss: 4.4173, Time: 0m3s


100%|██████████| 681/681 [00:02<00:00, 258.15it/s]


Epoch 49/50, Loss: 4.3816, Time: 0m2s


100%|██████████| 681/681 [00:02<00:00, 251.19it/s]

Epoch 50/50, Loss: 4.3527, Time: 0m2s


# Save the model and mapping

In [7]:
# Save the model
torch.save(model.state_dict(), "../models/noob.pth")

In [9]:
import pickle

with open("../models/heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)